In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [2]:
list=[[['B', 'A', 'A', 'C', 'B', 'A']],
       [['C', 'D', 'E']],
       [['B']],
       [['C', 'C', 'C']]]

df=spark.createDataFrame(list,['mylist'])

df.show()

+------------------+
 mylist|
+------------------+
[B, A, A, C, B, A]|
 [C, D, E]|
 [B]|
 [C, C, C]|
+------------------+

In [4]:
from pyspark.sql import functions as F
df\
  .withColumn("histo",\
          F.expr("""map_from_arrays(array_distinct(mylist),transform(array_distinct(mylist),\
              x-> size(filter(mylist,y-> y=x))))"""))\
  .show(truncate=False)

+------------------+------------------------+
mylist |histo |
+------------------+------------------------+
[B, A, A, C, B, A]|[B -> 2, A -> 3, C -> 1]|
[C, D, E] |[C -> 1, D -> 1, E -> 1]|
[B] |[B -> 1] |
[C, C, C] |[C -> 3] |
+------------------+------------------------+

In [5]:
list=[[9347774,'2019-01-01 00:00:00','2019-01-08 02:10:00', 7.090277777777778,    0.0],
      [9347774,'2019-01-08 02:10:00','2019-02-04 05:28:00',           27.1375,    0.0],
      [9347774,'2019-02-04 05:28:00','2019-03-05 19:29:00',29.584027777777777,    0.0],
      [9347774,'2019-03-05 19:29:00','2019-04-22 18:45:00', 47.96944444444444,    0.0],
      [9347774,'2019-04-22 18:45:00','2019-05-03 05:05:00',10.430555555555555,    0.0],
      [9347774,'2019-05-03 05:05:00','2019-05-17 16:25:00',14.472222222222221,    0.0],
      [9347774,'2019-05-17 16:25:00','2019-06-05 22:18:00', 19.24513888888889,    0.0],
      [9347774,'2019-01-01 00:00:00','2020-01-01 00:00:00',             365.0,    365.0],
      [9347774,'2019-06-05 22:18:00','2020-01-01 00:00:00',209.07083333333333,    0.0]]

df=spark.createDataFrame(list,['id','begin','end','days','overlap'])

df.show()


+-------+-------------------+-------------------+------------------+-------+
 id| begin| end| days|overlap|
+-------+-------------------+-------------------+------------------+-------+
9347774|2019-01-01 00:00:00|2019-01-08 02:10:00| 7.090277777777778| 0.0|
9347774|2019-01-08 02:10:00|2019-02-04 05:28:00| 27.1375| 0.0|
9347774|2019-02-04 05:28:00|2019-03-05 19:29:00|29.584027777777777| 0.0|
9347774|2019-03-05 19:29:00|2019-04-22 18:45:00| 47.96944444444444| 0.0|
9347774|2019-04-22 18:45:00|2019-05-03 05:05:00|10.430555555555555| 0.0|
9347774|2019-05-03 05:05:00|2019-05-17 16:25:00|14.472222222222221| 0.0|
9347774|2019-05-17 16:25:00|2019-06-05 22:18:00| 19.24513888888889| 0.0|
9347774|2019-01-01 00:00:00|2020-01-01 00:00:00| 365.0| 365.0|
9347774|2019-06-05 22:18:00|2020-01-01 00:00:00|209.07083333333333| 0.0|
+-------+-------------------+-------------------+------------------+-------+

In [6]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window().partitionBy("id").orderBy("begin")
w2=Window().partitionBy("id","begin","end").orderBy("begin")
w3=Window().partitionBy("id","begin","end")
w4=Window().partitionBy("id","begin","end","maxrownum").orderBy("begin")
df.withColumn("seq", F.expr("""sequence(to_timestamp(begin), to_timestamp(end),interval 1 day)"""))\
  .withColumn('maxrownum', F.max(F.row_number().over(w2)).over(w3))\
  .withColumn('rowNum', F.row_number().over(w4))\
  .withColumn("seq1", F.expr("""flatten(filter(collect_list(seq) over\
                                (partition by id),x-> arrays_overlap(x,seq)==True and seq!=x))"""))\
  .withColumn("overlap", F.when(F.row_number().over(w1)==1, F.lit(0))\
              .when(F.size(F.array_intersect("seq","seq1"))!=0,F.size(F.array_intersect("seq","seq1"))-1)
              .when((F.col("maxrownum")!=1)&(F.col("rowNum")<F.col("maxrownum")),F.col("days"))\
              .when(F.col("maxrownum")==1,F.col("days"))\
              .otherwise(F.lit(0)))\
              .replace(1,0)\
         .orderBy("id","end").drop("seq","seq1","rowNum","maxrownum").show()

+-------+-------------------+-------------------+------------------+------------------+
 id| begin| end| days| overlap|
+-------+-------------------+-------------------+------------------+------------------+
9347774|2019-01-01 00:00:00|2019-01-08 02:10:00| 7.090277777777778| 7.0|
9347774|2019-01-08 02:10:00|2019-02-04 05:28:00| 27.1375| 27.1375|
9347774|2019-02-04 05:28:00|2019-03-05 19:29:00|29.584027777777777|29.584027777777777|
9347774|2019-03-05 19:29:00|2019-04-22 18:45:00| 47.96944444444444| 47.96944444444444|
9347774|2019-04-22 18:45:00|2019-05-03 05:05:00|10.430555555555555|10.430555555555555|
9347774|2019-05-03 05:05:00|2019-05-17 16:25:00|14.472222222222221|14.472222222222221|
9347774|2019-05-17 16:25:00|2019-06-05 22:18:00| 19.24513888888889| 19.24513888888889|
9347774|2019-01-01 00:00:00|2020-01-01 00:00:00| 365.0| 0.0|
9347774|2019-06-05 22:18:00|2020-01-01 00:00:00|209.07083333333333|209.07083333333333|
+-------+-------------------+-------------------+------------------+------------------+

In [7]:
list=[[7777777,'2019-01-05 01:00:00','2019-04-04 00:00:00',88.95833333333333],
[7777777,'2019-04-04 00:00:00','2019-07-11 00:00:00',             98.0],
[7777777,'2019-07-11 00:00:00','2019-09-17 00:00:00',             68.0],
[7777777,'2019-09-17 00:00:00','2019-09-19 22:01:00',2.917361111111111],
[7777777,'2019-09-19 22:01:00','2020-01-01 00:00:00',103.0826388888889],
[7777777,'2019-09-19 22:01:00','2020-01-01 00:00:00',103.0826388888889]]

df=spark.createDataFrame(list,['id','begin','end','days'])

df.show()


+-------+-------------------+-------------------+-----------------+
 id| begin| end| days|
+-------+-------------------+-------------------+-----------------+
7777777|2019-01-05 01:00:00|2019-04-04 00:00:00|88.95833333333333|
7777777|2019-04-04 00:00:00|2019-07-11 00:00:00| 98.0|
7777777|2019-07-11 00:00:00|2019-09-17 00:00:00| 68.0|
7777777|2019-09-17 00:00:00|2019-09-19 22:01:00|2.917361111111111|
7777777|2019-09-19 22:01:00|2020-01-01 00:00:00|103.0826388888889|
7777777|2019-09-19 22:01:00|2020-01-01 00:00:00|103.0826388888889|
+-------+-------------------+-------------------+-----------------+

In [8]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window().partitionBy("id").orderBy("begin")
w2=Window().partitionBy("id","begin","end").orderBy("begin")
w3=Window().partitionBy("id","begin","end")
w4=Window().partitionBy("id","begin","end","maxrownum").orderBy("begin")
df.withColumn("seq", F.expr("""sequence(to_timestamp(begin), to_timestamp(end),interval 1 day)"""))\
  .withColumn('maxrownum', F.max(F.row_number().over(w2)).over(w3))\
  .withColumn('rowNum', F.row_number().over(w4))\
  .withColumn("seq1", F.expr("""flatten(filter(collect_list(seq) over\
                                (partition by id),x-> arrays_overlap(x,seq)==True and seq!=x))"""))\
  .withColumn("overlap", F.when(F.row_number().over(w1)==1, F.lit(0))\
              .when(F.size(F.array_intersect("seq","seq1"))!=0,F.size(F.array_intersect("seq","seq1"))-1)
              .when((F.col("maxrownum")!=1)&(F.col("rowNum")<F.col("maxrownum")),F.col("days"))\
              .when(F.col("maxrownum")==1,F.col("days"))\
              .otherwise(F.lit(0)))\
              .replace(1,0)\
         .orderBy("id","end").drop("seq","seq1","rowNum","maxrownum").show()

+-------+-------------------+-------------------+-----------------+-----------------+
 id| begin| end| days| overlap|
+-------+-------------------+-------------------+-----------------+-----------------+
7777777|2019-01-05 01:00:00|2019-04-04 00:00:00|88.95833333333333| 0.0|
7777777|2019-04-04 00:00:00|2019-07-11 00:00:00| 98.0| 0.0|
7777777|2019-07-11 00:00:00|2019-09-17 00:00:00| 68.0| 0.0|
7777777|2019-09-17 00:00:00|2019-09-19 22:01:00|2.917361111111111| 0.0|
7777777|2019-09-19 22:01:00|2020-01-01 00:00:00|103.0826388888889|103.0826388888889|
7777777|2019-09-19 22:01:00|2020-01-01 00:00:00|103.0826388888889| 0.0|
+-------+-------------------+-------------------+-----------------+-----------------+

In [9]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window().partitionBy("id").orderBy("begin")
w2=Window().partitionBy("id","begin","end").orderBy("begin")
w3=Window().partitionBy("id","begin","end")
w4=Window().partitionBy("id","begin","end","maxrownum").orderBy("begin")
df.withColumn("seq", F.expr("""sequence(to_timestamp(begin), to_timestamp(end),interval 1 day)"""))\
  .withColumn('maxrownum', F.max(F.row_number().over(w2)).over(w3))\
  .withColumn('rowNum', F.row_number().over(w4))\
  .withColumn("seq1", F.expr("""flatten(filter(collect_list(seq) over\
                                (partition by id order by begin),x-> arrays_overlap(x,seq)==True and seq!=x))"""))\
  .withColumn("overlap", F.when(F.row_number().over(w1)==1, F.lit(0))\
              .when(F.size(F.array_intersect("seq","seq1"))!=0,F.size(F.array_intersect("seq","seq1"))-1)
              .when((F.col("maxrownum")!=1)&(F.col("rowNum")<F.col("maxrownum")),F.col("days"))\
              .otherwise(F.lit(0)))\
         .orderBy("id","end").drop("seq","seq1","maxrownum","rowNum").show()

+-------+-------------------+-------------------+-----------------+-----------------+
 id| begin| end| days| overlap|
+-------+-------------------+-------------------+-----------------+-----------------+
7777777|2019-01-05 01:00:00|2019-04-04 00:00:00|88.95833333333333| 0.0|
7777777|2019-04-04 00:00:00|2019-07-11 00:00:00| 98.0| 0.0|
7777777|2019-07-11 00:00:00|2019-09-17 00:00:00| 68.0| 0.0|
7777777|2019-09-17 00:00:00|2019-09-19 22:01:00|2.917361111111111| 0.0|
7777777|2019-09-19 22:01:00|2020-01-01 00:00:00|103.0826388888889|103.0826388888889|
7777777|2019-09-19 22:01:00|2020-01-01 00:00:00|103.0826388888889| 0.0|
+-------+-------------------+-------------------+-----------------+-----------------+

In [10]:
+-------+-------------------+-------------------+-----------------+-------+
|     id|              begin|                end|             days|overlap|
+-------+-------------------+-------------------+-----------------+-------+
|7777777|2019-01-05 01:00:00|2019-04-04 00:00:00|88.95833333333333|      0|
|7777777|2019-04-04 00:00:00|2019-07-11 00:00:00|             98.0|      0|
|7777777|2019-07-11 00:00:00|2019-09-17 00:00:00|             68.0|      0|
|7777777|2019-09-17 00:00:00|2019-09-19 22:01:00|2.917361111111111|      0|
|7777777|2019-09-19 22:01:00|2020-01-01 00:00:00|103.0826388888889|103.082|
|7777777|2019-09-19 22:01:00|2020-01-01 00:00:00|103.0826388888889|      0|
+-------+-------------------+-------------------+-----------------+-------+

In [11]:
list=[[2      ,'2019-01-01 00:00:00','2019-12-25 00:00:00',358.0],   
[2      ,'2019-12-25 00:00:00','2020-01-01 00:00:00',  7.0],   
[3      ,'2019-01-01 00:00:00','2019-12-25 00:00:00',358.0],  
[3      ,'2019-12-20 00:00:00','2020-01-01 00:00:00', 12.0],    
[4      ,'2019-01-01 00:00:00','2019-12-25 00:00:00',358.0],  
[4      ,'2019-01-01 00:00:00','2019-11-25 00:00:00',328.0],
[4      ,'2019-12-20 00:00:00','2020-01-01 00:00:00', 12.0]]

df=spark.createDataFrame(list,['id','begin','end','days'])

df.show()


+---+-------------------+-------------------+-----+
 id| begin| end| days|
+---+-------------------+-------------------+-----+
 2|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|
 2|2019-12-25 00:00:00|2020-01-01 00:00:00| 7.0|
 3|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|
 3|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|
 4|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|
 4|2019-01-01 00:00:00|2019-11-25 00:00:00|328.0|
 4|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|
+---+-------------------+-------------------+-----+

In [12]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window().partitionBy("id").orderBy("begin")
w2=Window().partitionBy("id","begin","end").orderBy("begin")
w3=Window().partitionBy("id","begin","end")
w4=Window().partitionBy("id","begin","end","maxrownum").orderBy("begin")
df.withColumn("seq", F.expr("""sequence(to_timestamp(begin), to_timestamp(end),interval 1 day)"""))\
  .withColumn('maxrownum', F.max(F.row_number().over(w2)).over(w3))\
  .withColumn('rowNum', F.row_number().over(w4))\
  .withColumn("seq1", F.expr("""flatten(filter(collect_list(seq) over\
                                (partition by id),x-> arrays_overlap(x,seq)==True and seq!=x))"""))\
  .withColumn("overlap", F.when(F.row_number().over(w1)==1, F.lit(0))\
              .when(F.size(F.array_intersect("seq","seq1"))!=0,F.size(F.array_intersect("seq","seq1"))-1)
              .when((F.col("maxrownum")!=1)&(F.col("rowNum")<F.col("maxrownum")),F.col("days"))\
              .when(F.col("maxrownum")==1,F.col("days"))\
              .otherwise(F.lit(0)))\
              .replace(1,0)\
         .orderBy("id","end").drop("seq","seq1","rowNum","maxrownum").show()

In [13]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window().partitionBy("id").orderBy("begin")
w2=Window().partitionBy("id","begin","end").orderBy("begin")
w3=Window().partitionBy("id","begin","end")
w4=Window().partitionBy("id","begin","end","maxrownum").orderBy("begin")
df.withColumn("seq", F.expr("""sequence(to_timestamp(begin), to_timestamp(end),interval 1 day)"""))\
  .withColumn('maxrownum', F.max(F.row_number().over(w2)).over(w3))\
  .withColumn('rowNum', F.row_number().over(w4))\
  .withColumn("seq1", F.expr("""flatten(filter(collect_list(seq) over\
                                (partition by id),x-> arrays_overlap(x,seq)==True and seq!=x))"""))\
  .withColumn("overlap", F.when(F.row_number().over(w1)==1, F.lit(0))\
              .when(F.size(F.array_intersect("seq","seq1"))!=0,F.size(F.array_intersect("seq","seq1"))-1)
              .when((F.col("maxrownum")!=1)&(F.col("rowNum")<F.col("maxrownum")),F.col("days"))\
              .when(F.col("maxrownum")==1,F.col("days"))\
              .otherwise(F.lit(0)))\
              .replace(1,0)\
         .orderBy("id","end").drop("seq","seq1","rowNum","maxrownum").show()

+---+-------------------+-------------------+-----+-------+
 id| begin| end| days|overlap|
+---+-------------------+-------------------+-----+-------+
 2|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 0.0|
 2|2019-12-25 00:00:00|2020-01-01 00:00:00| 7.0| 0.0|
 3|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 0.0|
 3|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0| 5.0|
 4|2019-01-01 00:00:00|2019-11-25 00:00:00|328.0| 328.0|
 4|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 0.0|
 4|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0| 5.0|
+---+-------------------+-------------------+-----+-------+

In [14]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window().partitionBy("id").orderBy("begin")
w2=Window().partitionBy("id","begin","end").orderBy("begin")
w3=Window().partitionBy("id","begin","end")
w4=Window().partitionBy("id","begin","end","maxrownum").orderBy("begin")
df.withColumn("seq", F.expr("""sequence(to_timestamp(begin), to_timestamp(end),interval 1 day)"""))\
  .withColumn('maxrownum', F.max(F.row_number().over(w2)).over(w3))\
  .withColumn('rowNum', F.row_number().over(w4))\
  .withColumn("seq1", F.expr("""flatten(filter(collect_list(seq) over\
                                (partition by id),x-> arrays_overlap(x,seq)==True and seq!=x))"""))\
  .withColumn("overlap", F.when(F.row_number().over(w1)==1, F.lit(0))\
              .when(F.size(F.array_intersect("seq","seq1"))!=0,F.size(F.array_intersect("seq","seq1"))-1)
              .when((F.col("maxrownum")!=1)&(F.col("rowNum")<F.col("maxrownum")),F.col("days"))\
              .when(F.col("maxrownum")==1,F.col("days"))\
              .otherwise(F.lit(0)))\
              .replace(1,0)\
         .orderBy("id","end").drop("seq","seq1","rowNum","overlap").show()

+---+-------------------+-------------------+-----+---------+------+-------+
 id| begin| end| days|maxrownum|rowNum|overlap|
+---+-------------------+-------------------+-----+---------+------+-------+
 2|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 0| 0| 0.0|
 2|2019-12-25 00:00:00|2020-01-01 00:00:00| 7.0| 0| 0| 0.0|
 3|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 0| 0| 0.0|
 3|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0| 0| 0| 5.0|
 4|2019-01-01 00:00:00|2019-11-25 00:00:00|328.0| 0| 0| 328.0|
 4|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 0| 0| 0.0|
 4|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0| 0| 0| 5.0|
+---+-------------------+-------------------+-----+---------+------+-------+

In [15]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window().partitionBy("id").orderBy("begin")
w2=Window().partitionBy("id","begin","end").orderBy("begin")
w3=Window().partitionBy("id","begin","end")
w4=Window().partitionBy("id","begin","end","maxrownum").orderBy("begin")
df.withColumn("seq", F.expr("""sequence(to_timestamp(begin), to_timestamp(end),interval 1 day)"""))\
  .withColumn('maxrownum', F.max(F.row_number().over(w2)).over(w3))\
  .withColumn('rowNum', F.row_number().over(w4))\
  .withColumn("seq1", F.expr("""flatten(filter(collect_list(seq) over\
                                (partition by id order by begin),x-> arrays_overlap(x,seq)==True and seq!=x))"""))\
  .withColumn("overlap", F.when(F.row_number().over(w1)==1, F.lit(0))\
              .when(F.size(F.array_intersect("seq","seq1"))!=0,F.size(F.array_intersect("seq","seq1"))-1)
              .when((F.col("maxrownum")!=1)&(F.col("rowNum")<F.col("maxrownum")),F.col("days"))\
              .otherwise(F.lit(0)))\
         .orderBy("id","end").drop("seq","seq1","maxrownum","rowNum").show()

+---+-------------------+-------------------+-----+-------+
 id| begin| end| days|overlap|
+---+-------------------+-------------------+-----+-------+
 2|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 0.0|
 2|2019-12-25 00:00:00|2020-01-01 00:00:00| 7.0| 0.0|
 3|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 0.0|
 3|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0| 5.0|
 4|2019-01-01 00:00:00|2019-11-25 00:00:00|328.0| 328.0|
 4|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 0.0|
 4|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0| 5.0|
+---+-------------------+-------------------+-----+-------+

In [16]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window().partitionBy("id").orderBy("begin")
df.withColumn("seq", F.expr("""sequence(to_timestamp(begin), to_timestamp(end),interval 1 day)"""))\
  .withColumn("seq1", F.expr("""flatten(filter(collect_list(seq) over\
                                (partition by id),x-> arrays_overlap(x,seq)==True and seq!=x))"""))\
  .withColumn("overlap", F.when(F.row_number().over(w1)==1, F.lit(0))\
              .otherwise(F.size(F.array_intersect("seq","seq1"))-1)).orderBy("id","end").drop("seq","seq1").show()

+---+-------------------+-------------------+-----+-------+
 id| begin| end| days|overlap|
+---+-------------------+-------------------+-----+-------+
 2|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 0|
 2|2019-12-25 00:00:00|2020-01-01 00:00:00| 7.0| 0|
 3|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 0|
 3|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0| 5|
 4|2019-01-01 00:00:00|2019-11-25 00:00:00|328.0| 328|
 4|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 0|
 4|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0| 5|
+---+-------------------+-------------------+-----+-------+

In [17]:
df.show() #sample dataframe
#+---+-------------------+-------------------+-----+
#| id|              begin|                end| days|
#+---+-------------------+-------------------+-----+
#|  2|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|
#|  2|2019-12-25 00:00:00|2020-01-01 00:00:00|  7.0|
#|  3|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|
#|  3|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|
#|  4|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|
#|  4|2019-01-01 00:00:00|2019-11-25 00:00:00|328.0|
#|  4|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|
#+---+-------------------+-------------------+-----+

from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window().partitionBy("id").orderBy("begin")
df.withColumn("seq", F.expr("""sequence(to_timestamp(begin), to_timestamp(end),interval 1 day)"""))\
  .withColumn("seq1", F.expr("""flatten(filter(collect_list(seq) over\
                                (partition by id),x-> arrays_overlap(x,seq)==True and seq!=x))"""))\
  .withColumn("overlap", F.size(F.array_intersect("seq","seq1"))-1).orderBy("id","end").drop("seq","seq1").show()

#+---+-------------------+-------------------+-----+-------+
#| id|              begin|                end| days|overlap|
#+---+-------------------+-------------------+-----+-------+
#|  2|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|      0|
#|  2|2019-12-25 00:00:00|2020-01-01 00:00:00|  7.0|      0|
#|  3|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|      0|
#|  3|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|      5|
#|  4|2019-01-01 00:00:00|2019-11-25 00:00:00|328.0|    328|
#|  4|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|      0|
#|  4|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|      5|
#+---+-------------------+-------------------+-----+-------+

+---+-------------------+-------------------+-----+
 id| begin| end| days|
+---+-------------------+-------------------+-----+
 2|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|
 2|2019-12-25 00:00:00|2020-01-01 00:00:00| 7.0|
 3|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|
 3|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|
 4|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|
 4|2019-01-01 00:00:00|2019-11-25 00:00:00|328.0|
 4|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|
+---+-------------------+-------------------+-----+

+---+-------------------+-------------------+-----+-------+
 id| begin| end| days|overlap|
+---+-------------------+-------------------+-----+-------+
 2|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 0|
 2|2019-12-25 00:00:00|2020-01-01 00:00:00| 7.0| 0|
 3|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 5|
 3|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0| 5|
 4|2019-01-01 00:00:00|2019-11-25 00:00:00|328.0| 328|
 4|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 334|
 4|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0| 5|
+---+-------------------+-------------------+-----+-------+

In [18]:
df.show() #sample dataframe
#+---+-------------------+-------------------+-----+
#| id|              begin|                end| days|
#+---+-------------------+-------------------+-----+
#|  2|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|
#|  2|2019-12-25 00:00:00|2020-01-01 00:00:00|  7.0|
#|  3|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|
#|  3|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|
#|  4|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|
#|  4|2019-01-01 00:00:00|2019-11-25 00:00:00|328.0|
#|  4|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|
#+---+-------------------+-------------------+-----+

from pyspark.sql import functions as F
from pyspark.sql.window import Window

w1=Window().partitionBy("id").orderBy("begin")
df.withColumn("seq", F.expr("""sequence(to_timestamp(begin), to_timestamp(end),interval 1 day)"""))\
  .withColumn("seq1", F.expr("""flatten(filter(collect_list(seq) over\
                                (partition by id),x-> arrays_overlap(x,seq)==True and seq!=x))"""))\
  .withColumn("overlap", F.when(F.row_number().over(w1)==1, F.lit(0))\
              .otherwise(F.size(F.array_intersect("seq","seq1"))-1)).orderBy("id","end").drop("seq","seq1").show()

#+---+-------------------+-------------------+-----+-------+
#| id|              begin|                end| days|overlap|
#+---+-------------------+-------------------+-----+-------+
#|  2|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|      0|
#|  2|2019-12-25 00:00:00|2020-01-01 00:00:00|  7.0|      0|
#|  3|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|      0|
#|  3|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|      5|
#|  4|2019-01-01 00:00:00|2019-11-25 00:00:00|328.0|    328|
#|  4|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|      0|
#|  4|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|      5|
#+---+-------------------+-------------------+-----+-------+


+---+-------------------+-------------------+-----+
 id| begin| end| days|
+---+-------------------+-------------------+-----+
 2|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|
 2|2019-12-25 00:00:00|2020-01-01 00:00:00| 7.0|
 3|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|
 3|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|
 4|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|
 4|2019-01-01 00:00:00|2019-11-25 00:00:00|328.0|
 4|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|
+---+-------------------+-------------------+-----+

+---+-------------------+-------------------+-----+-------+
 id| begin| end| days|overlap|
+---+-------------------+-------------------+-----+-------+
 2|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 0|
 2|2019-12-25 00:00:00|2020-01-01 00:00:00| 7.0| 0|
 3|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 0|
 3|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0| 5|
 4|2019-01-01 00:00:00|2019-11-25 00:00:00|328.0| 328|
 4|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0| 0|
 4|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0| 5|
+---+-------------------+-------------------+-----+-------+

In [19]:
w=Window().partitionBy("id")
w1=Window().partitionBy("id").orderBy("begin")
df.withColumn("seq", F.expr("""sequence(to_timestamp(begin), to_timestamp(end),interval 1 day)"""))\
  .withColumn("seq1", F.collect_list("seq").over(w))\
  .withColumn("seq1", F.expr("""flatten(filter(collect_list(seq) over\
                                (partition by id order by begin),x-> arrays_overlap(x,seq)==True and seq!=x))"""))\
  .withColumn("intersect", F.when(F.row_number().over(w1)==1, F.lit(0))\
              .otherwise(F.size(F.array_intersect("seq","seq1"))-1)).orderBy("id","end").show()

In [20]:
collect_list(wpi)\
                                    over (order by financial_year)

In [21]:
+-------+-------------------+-------------------+-----+-----+
|  id   |              begin|                end| days| over|
+-------+-------------------+-------------------+-----+-----+
|2      |2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|    0|
|2      |2019-12-25 00:00:00|2020-01-01 00:00:00|  7.0|    0|
+-------+-------------------+-------------------+-----+-----+
+-------+-------------------+-------------------+-----+-----+
|     id|              begin|                end| days| over|
+-------+-------------------+-------------------+-----+-----+
|3      |2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|    0|
|3      |2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|    5|
+-------+-------------------+-------------------+-----+-----+
+-------+-------------------+-------------------+-----+-----+
|     id|              begin|                end| days| over|
+-------+-------------------+-------------------+-----+-----+
|4      |2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|    0|
|4      |2019-01-01 00:00:00|2019-11-25 00:00:00|328.0|328.0|
|4      |2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|    5|
+-------+-------------------+-------------------+-----+-----+

In [22]:
w=Window().partitionBy("id").orderBy("begin")

df.withColumn("rowNum", F.row_number().over(w))\
  .withColumn("dates", F.when(F.col("rowNum")%2==0, F.col("begin")).otherwise(F.col("end")))\
  .withColumn("overlap", F.when(F.col("rowNum")==1, F.lit(0))\
                          .otherwise(F.datediff(F.col("dates"),F.lag("dates").over(w)))).drop("rowNum").show()

+---+-------------------+-------------------+-----+-------------------+-------+
 id| begin| end| days| dates|overlap|
+---+-------------------+-------------------+-----+-------------------+-------+
 3|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|2019-12-25 00:00:00| 0|
 3|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|2019-12-20 00:00:00| -5|
 2|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|2019-12-25 00:00:00| 0|
 2|2019-12-25 00:00:00|2020-01-01 00:00:00| 7.0|2019-12-25 00:00:00| 0|
 4|2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|2019-12-25 00:00:00| 0|
 4|2019-01-01 00:00:00|2019-11-25 00:00:00|328.0|2019-01-01 00:00:00| -358|
 4|2019-12-20 00:00:00|2020-01-01 00:00:00| 12.0|2020-01-01 00:00:00| 365|
+---+-------------------+-------------------+-----+-------------------+-------+

In [23]:
w=Window().partitionBy("id").orderBy("begin")

df.withColumn("rowNum", F.row_number().over(w))\
  .withColumn("dates", F.when(F.col("rowNum")==1, F.array(F.lit(1))).otherwise(F.array(F.col("begin"),F.lag("end").over(w))))\
  .withColumn("over",F.when(F.size("dates")==1,F.lit(0)).otherwise(F.datediff(F.col("dates")[1],F.col("dates")[0])))\
  .show(truncate=False)

+---+-------------------+-------------------+-----+------+------------------------------------------+----+
id |begin |end |days |rowNum|dates |over|
+---+-------------------+-------------------+-----+------+------------------------------------------+----+
3 |2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|1 |[1] |0 |
3 |2019-12-20 00:00:00|2020-01-01 00:00:00|12.0 |2 |[2019-12-20 00:00:00, 2019-12-25 00:00:00]|5 |
2 |2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|1 |[1] |0 |
2 |2019-12-25 00:00:00|2020-01-01 00:00:00|7.0 |2 |[2019-12-25 00:00:00, 2019-12-25 00:00:00]|0 |
4 |2019-01-01 00:00:00|2019-12-25 00:00:00|358.0|1 |[1] |0 |
4 |2019-01-01 00:00:00|2019-11-25 00:00:00|328.0|2 |[2019-01-01 00:00:00, 2019-12-25 00:00:00]|358 |
4 |2019-12-20 00:00:00|2020-01-01 00:00:00|12.0 |3 |[2019-12-20 00:00:00, 2019-11-25 00:00:00]|-25 |
+---+-------------------+-------------------+-----+------+------------------------------------------+----+

In [27]:
import pandas as pd
# Master DF
df = pd.DataFrame({"Name": ["Mike", "Bob", "Steve", "Jim", "Dan"], "Age": [22, 44, 66, 22, 66], "Job": ["Doc", "Cashier", "Fireman", "Doc", "Fireman"]})
df=spark.createDataFrame(df)
#Secondary DF
df1 = pd.DataFrame({"Age": [22, 66], "Job": ["Doc", "Fireman"]})
df1=spark.createDataFrame(df1)
new_df1 = pd.DataFrame({"Age": [22, 66], "Job": ["Doc", "Fireman"], "Returned_value": ['Mike-Jim', 'Steve-Dan']})
new_df1=spark.createDataFrame(new_df1)
df.show()
df1.show()
new_df1.show()

+-----+---+-------+
 Name|Age| Job|
+-----+---+-------+
 Mike| 22| Doc|
 Bob| 44|Cashier|
Steve| 66|Fireman|
 Jim| 22| Doc|
 Dan| 66|Fireman|
+-----+---+-------+

+---+-------+
Age| Job|
+---+-------+
 22| Doc|
 66|Fireman|
+---+-------+

+---+-------+--------------+
Age| Job|Returned_value|
+---+-------+--------------+
 22| Doc| Mike-Jim|
 66|Fireman| Steve-Dan|
+---+-------+--------------+

In [28]:
from pyspark.sql import functions as F

df.join(df1,['Age','Job'])\
  .groupBy("Age","Job").agg(F.concat_ws('-',F.collect_list("Name")).alias("Returned_value")).show()

#+---+-------+--------------+
#|Age|    Job|Returned_value|
#+---+-------+--------------+
#| 22|    Doc|      Mike-Jim|
#| 66|Fireman|     Steve-Dan|
#+---+-------+--------------+

+---+-------+--------------+
Age| Job|Returned_value|
+---+-------+--------------+
 22| Doc| Mike-Jim|
 66|Fireman| Steve-Dan|
+---+-------+--------------+

In [29]:
list=[[ 1  , '12:00:00'    , 'a'],
      [ 2  , '13:00:00'    , 'b'],
      [ 3  , '11:00:00'    , 'c']]

df1=spark.createDataFrame(list,['id','SMS Created','Content'])

list1=[[ 'Created' , '11:30:00' , 1  ,[1,2]],
    [ 'Updated' , '11:42:00' , 1  ,[1,2,3]],
[ 'Updated' , '11:50:00' , 1  ,[1,2,4]],
[ 'Updated' , '12:50:00' , 1  ,[1,2]],
[ 'Created' , '12:30:00' , 2 ,[1,2]],
[ 'Updated' , '12:42:00' , 2  ,[1,2,3]],
[ 'Updated' , '12:50:00' , 2  ,[1,2,4]],
[ 'Updated' , '13:10:00' , 2  ,[1,2]],
[ 'Created' , '10:30:00' , 3  ,[1,2]],
[ 'Updated' , '10:42:00' , 3  ,[1,2,3]],
[ 'Updated' , '10:50:00' , 3  ,[1,2,4]],
[ 'Updated' , '12:10:00' , 2  ,[1,2]]]

df2=spark.createDataFrame(list1,['Event','Time','id','Members'])

df1.show()

df2.show()

+---+-----------+-------+
 id|SMS Created|Content|
+---+-----------+-------+
 1| 12:00:00| a|
 2| 13:00:00| b|
 3| 11:00:00| c|
+---+-----------+-------+

+-------+--------+---+---------+
 Event| Time| id| Members|
+-------+--------+---+---------+
Created|11:30:00| 1| [1, 2]|
Updated|11:42:00| 1|[1, 2, 3]|
Updated|11:50:00| 1|[1, 2, 4]|
Updated|12:50:00| 1| [1, 2]|
Created|12:30:00| 2| [1, 2]|
Updated|12:42:00| 2|[1, 2, 3]|
Updated|12:50:00| 2|[1, 2, 4]|
Updated|13:10:00| 2| [1, 2]|
Created|10:30:00| 3| [1, 2]|
Updated|10:42:00| 3|[1, 2, 3]|
Updated|10:50:00| 3|[1, 2, 4]|
Updated|12:10:00| 2| [1, 2]|
+-------+--------+---+---------+

In [30]:
w=Window().partitionBy("id")
cond=F.expr("""`SMS Created`>Time""")
df1.join(df2,['id'])\
   .withColumn("max", F.max("Time").over(w)).filter('max=Time').drop("max").orderBy("id").show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o285.join.
: org.apache.spark.sql.AnalysisException: USING column ``SMS Created`>Time` cannot be resolved on the left side of the join. The left-side columns: [id, SMS Created, Content];
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$100$$anonfun$apply$71.apply(Analyzer.scala:2364)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$100$$anonfun$apply$71.apply(Analyzer.scala:2364)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$100.apply(Analyzer.scala:2363)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$100.apply(Analyzer.scala:2362)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.Iterator$class.foreach(Iterator.scala:891)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1334)
	at scala.collection.IterableLike$class.foreach(IterableLike.scala:72)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:54)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.org$apache$spark$sql$catalyst$analysis$Analyzer$$commonNaturalJoinProcessing(Analyzer.scala:2362)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveNaturalAndUsingJoin$$anonfun$apply$34.applyOrElse(Analyzer.scala:2312)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveNaturalAndUsingJoin$$anonfun$apply$34.applyOrElse(Analyzer.scala:2309)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$apply$1.apply(AnalysisHelper.scala:90)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$apply$1.apply(AnalysisHelper.scala:90)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:89)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsUp(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsUp(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveNaturalAndUsingJoin$.apply(Analyzer.scala:2309)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveNaturalAndUsingJoin$.apply(Analyzer.scala:2308)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:112)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:109)
	at scala.collection.LinearSeqOptimized$class.foldLeft(LinearSeqOptimized.scala:124)
	at scala.collection.immutable.List.foldLeft(List.scala:84)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:109)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:101)
	at scala.collection.immutable.List.

In [31]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy("id")
df1.join(df2.withColumnRenamed("id","id2"), (F.col("id")==F.col("id2"))&(F.col("SMS Created")>F.col("Time"))).drop("id2")\
   .withColumn("max", F.max("Time").over(w))\
   .filter('max=Time').drop("max").orderBy("id").show()

+---+-----------+-------+-------+--------+---------+
 id|SMS Created|Content| Event| Time| Members|
+---+-----------+-------+-------+--------+---------+
 1| 12:00:00| a|Updated|11:50:00|[1, 2, 4]|
 2| 13:00:00| b|Updated|12:50:00|[1, 2, 4]|
 3| 11:00:00| c|Updated|10:50:00|[1, 2, 4]|
+---+-----------+-------+-------+--------+---------+

In [32]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy("id")
df1.join(df2, ['id']&(df1['SMS Created']>df2.Time))\
   .withColumn("max", F.max("Time").over(w))\
   .filter('max=Time').drop("max").orderBy("id").show()

--------------------------------------------------------------------------- 
 Py4JError Traceback (most recent call last)
 <command-1584831649314246> in <module> 
 3 
 4 w = Window ( ) . partitionBy ( "id" ) 
 ----> 5 df1 . join ( df2 , [ 'id' ] & ( df1 [ 'SMS Created' ] > df2 . Time ) ) \ 
 6 . withColumn ( "max" , F . max ( "Time" ) . over ( w ) ) \ 
 7 . filter ( 'max=Time' ) . drop ( "max" ) . orderBy ( "id" ) . show ( ) 

 /databricks/spark/python/pyspark/sql/column.py in _ (self, other) 
 113 def _ ( self , other ) : 
 114 jc = other . _jc if isinstance ( other , Column ) else other
 --> 115 njc = getattr ( self . _jc , name ) ( jc ) 
 116 return Column ( njc ) 
 117 _ . __doc__ = doc

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 330 raise Py4JError(
 331 "An error occurred while calling {0}{1}{2}. Trace:\n{3}\n" . 
 --> 332 format(target_id, ".", name, value))
 333 else : 
 334 raise Py4JError(

 Py4JError : An error occurred while calling o2367.and. Trace:
py4j.Py4JException: Method and([class java.util.ArrayList]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:341)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:349)
	at py4j.Gateway.invoke(Gateway.java:286)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [33]:
from pyspark.sql import *

department1 = Row(id='123456', name='Computer Science')
department2 = Row(id='789012', name='Mechanical Engineering')

Employee = Row("firstName", "lastName", "email", "salary")
employee1 = Employee('michael', 'armbrust', 'no-reply@berkeley.edu', 100000)
employee2 = Employee('xiangrui', 'meng', 'no-reply@stanford.edu', 120000)


departmentWithEmployees1 = Row(department=department1, employees=[employee1, employee2])
departmentWithEmployees2 = Row(department=department2, employees=[employee1, employee2])


departmentsWithEmployeesSeq1 = [departmentWithEmployees1, departmentWithEmployees2]
df1 = spark.createDataFrame(departmentsWithEmployeesSeq1)

df1.show(truncate=False)
df1.printSchema()

+--------------------------------+-----------------------------------------------------------------------------------------------------+
department |employees |
+--------------------------------+-----------------------------------------------------------------------------------------------------+
[123456, Computer Science] |[[michael, armbrust, no-reply@berkeley.edu, 100000], [xiangrui, meng, no-reply@stanford.edu, 120000]]|
[789012, Mechanical Engineering]|[[michael, armbrust, no-reply@berkeley.edu, 100000], [xiangrui, meng, no-reply@stanford.edu, 120000]]|
+--------------------------------+-----------------------------------------------------------------------------------------------------+

root
-- department: struct (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
-- employees: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- firstName: string (nullable = true)
 | |-- lastName: string (nullable = true)
 | |-- email: string (nullable = true)
 | |-- salary: long (nullable = true)

In [34]:
df1.schema.fields[1]

Out[34]: StructField(employees,ArrayType(StructType(List(StructField(firstName,StringType,true),StructField(lastName,StringType,true),StructField(email,StringType,true),StructField(salary,LongType,true))),true),true)

In [35]:
from pyspark.sql import functions as F

df1.selectExpr("department","""inline_outer(employees)""")\
   .withColumn("FullName", F.concat_ws(" ","firstName","lastName"))\
   .select("department", F.array(F.struct(*[F.col(x).alias(x) for x in\
                                     ['firstName','lastName','email','salary','FullName']]))\
           .alias("employees")).printSchema()

root
-- department: struct (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
-- employees: array (nullable = false)
 |-- element: struct (containsNull = false)
 | |-- firstName: string (nullable = true)
 | |-- lastName: string (nullable = true)
 | |-- email: string (nullable = true)
 | |-- salary: long (nullable = true)
 | |-- FullName: string (nullable = false)

In [36]:
df1.withColumn("ar", F.expr("""transform(employees, x-> struct(concat_ws(' ',x.firstName,x.lastName) as\
                               FullName))"""))\
   .withColumn("employees", F.arrays_zip("employees","ar"))\
   .withColumn("employees", F.expr("""transform(employees,x->\
    struct(x.employees.firstName))"""))\
   .drop("ar").printSchema()

In [37]:
root
 |-- department: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- employees: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- firstName: string (nullable = true)
 |    |    |-- lastName: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- salary: long (nullable = true)
 |    |    |-- FullName: string (containsNull = true)l